<a href="https://colab.research.google.com/github/LittleAri/ALICE/blob/main/ALICE_rCNN_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mask RCNN Training for ALICE

## Bad Label Detection

Train model ([Mask RCNN](https://github.com/matterport/Mask_RCNN)) on good and bad merged labels. This is multiclass classification.

This notebook is a copy of the notebook used to train the CNN to find labels in images, except the aim of this one is to detect bad labels.

#### Step 0
Mount Google Drive.

This step is useful if using Google Colab, as it's often easier to save documents there.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Step 1

Check that images have loaded properly.

This is a precautionary step which is helpful when using Google Colab, to check if images have been uploaded correctly.

In [ ]:
import os
import skimage.io as io

In [ ]:
errors = 0
non_errors = 0

for nm in os.listdir("drive/My Drive/ALICE/balloon/train"):
  if "json" not in nm:
    try:
      image = io.imread("drive/My Drive/ALICE/balloon/train/"+nm)
      non_errors = non_errors+1
    except:
      print(nm)
      errors = errors+1

print(["val",non_errors,errors])

['val', 65, 0]


In [ ]:
errors = 0
non_errors = 0

for nm in os.listdir("balloon/train"):
  if "json" not in nm:
    try:
      image = io.imread("balloon/train/"+nm)
      non_errors = non_errors+1
    except:
      print(nm)
      errors = errors+1

print(["val",non_errors,errors])

['val', 456, 0]


#### Step 2

Import certain versions of Python packages.

In [ ]:
!pip install tensorflow==1.15.5
!pip install keras==2.1.0
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install numpy==1.21.5
!pip install tensorflow-gpu==1.15.5
!pip install scikit-image==0.16.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 1.6 kB/s 
     |████████████████████████████████| 3.8 MB 42.4 MB/s 
     |████████████████████████████████| 2.9 MB 55.0 MB/s 
     |████████████████████████████████| 20.1 MB 1.0 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
     |████████████████████████████████| 503 kB 70.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=89c75ed0f094c202c3c33de1fae0a675f3569512046e14bfaa2441bbf996f02c
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninst

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 302 kB 13.6 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-empk1w_o
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-empk1w_o
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=5576ead410b94d1872f3277fd2e1dba505044e852571cb1c4c5ae3145e174a5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-l05shzeo/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 411.0 MB 24 kB/s 
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.18.5 which is incompatible.
tensorflow-probability 0.16.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.18.5 which is incompatible.
plotnine 0.8.0 requires numpy>=1.19.0, but you have numpy 1.18.5 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.5 which is 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.5 MB 1.2 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3


#### Step 3:
Import libraries.

In [ ]:
import os
import sys
import json
import re
import random
import numpy as np
import matplotlib.pyplot as plt
from mrcnn import *
# import utils
# import  visualize
from mrcnn.visualize import display_images
import final
#from model import log
import skimage
#from google.colab import files as files_
import os
import tensorflow as tf
%matplotlib inline 

#### Step 4

Double check if there's a GPU.

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


#### Step 5

Train model by building on top of pre-trained weights.

The path for the images / labels that will be used for trianing, should be specified in _folder_for_labels_. The path to the folder containing the model and the pre-trained weights should be specified in _folder_for_model_.

In [ ]:
!python3 final.py train --dataset=drive/MyDrive/ALICE/balloon --weights=coco

Using TensorFlow backend.
Weights:  coco
Dataset:  drive/MyDrive/ALICE/balloon
Logs:  /logs

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bb